In [1]:
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
import pandas as pd
import numpy as np
import requests
import time

In [ ]:
def get_team_df(dict_correspondances, countries):
    player_name, player_country, player_position, player_age, player_description, player_link = [], [], [], [], [], []
    browser = webdriver.Firefox()
    browser.get("https://www.fifa.com/worldcup/teams/")
    time.sleep(1)
    
    for country in countries:
        element = browser.find_elements_by_xpath("//div[@class='fi-team-card__flag']")[dict_correspondances[country]]
        browser.execute_script("arguments[0].click();", element)
        time.sleep(1)

        for p in range(23):
            player = browser.find_elements_by_xpath("//div[@class='fi-p']/a[@class='fi-p--link']")[p]
            new_url = player.get_attribute('href')
            browser.get(new_url)
            time.sleep(1)
            player_name.append(browser.find_element_by_xpath("//div[@class='fi-p__name']").text)
            print(player_name[-1])
            player_country.append(country)
            player_position.append(browser.find_element_by_xpath("//div[@class='fi-p__role']").text)
            player_age.append(browser.find_element_by_xpath("//div[@class='fi-p__profile-number__number']").text)
            try:
                player_description.append(browser.find_element_by_xpath("//div[@class='fi-p__profile-text']").text)
            except NoSuchElementException:
                player_description.append("")
            my_img = browser.find_elements_by_tag_name('image')
            for image in my_img:
                link = image.get_attribute('xlink:href')
                player_link.append(link)
            browser.back()
            time.sleep(1)

        browser.get("https://www.fifa.com/worldcup/teams/")
        time.sleep(2)
    
    df = pd.DataFrame({"Name":player_name, "Country":player_country, "Position":player_position, "Age":player_age, 
                   "Description":player_description, "Link":player_link})
    df["Firstname"] = df["Name"].apply(lambda x: x.split()[0].capitalize())
    df["Lastname"] = df["Name"].apply(lambda x: "".join(x.split()[1:]).capitalize())
    return df

dict_correspondances = {"Australie": 0, "Belgique": 2, "Bresil": 3,
                        "Colombie": 4, "Costa": 5, "Croatie": 6,
                        "Danemark": 7, "Egypte": 8, "Angleterre": 9,
                        "France": 10, "Allemagne": 11, "Iran": 13,
                        "Maroc": 17, "Perou": 20, "Pologne": 21,
                        "Portugal": 22, "Russie": 23, "Arabie": 24,
                        "Serbie": 26, "Espagne": 27, "Suede": 28,
                        "Suisse": 29, "Tunisie": 30}

df = get_team_df(dict_correspondances, ["France"])

HUGO LLORIS
STEVE MANDANDA
ALPHONSE AREOLA
BENJAMIN PAVARD
PRESNEL KIMPEMBE
RAPHAEL VARANE
SAMUEL UMTITI


In [3]:
df.shape

(184, 8)

In [4]:
def image_downloader(name, team, link):
    with open(team.lower() + '/' + name.capitalize() + '.jpg', 'wb') as handle:
            response = requests.get(link, stream=True)

            if not response.ok:
                print(response)

            for block in response.iter_content(1024):
                if not block:
                    break

                handle.write(block)

for n, t, l in zip(df["Lastname"].tolist(), df["Country"].tolist(), df["Link"].tolist()):
    image_downloader(n, t, l)

In [5]:
df.to_csv("df_RUS_ARA_IRA_MAR_PER_POR_ESP.csv", index=False)